Fit CNN in PyTorch
===


<div class="alert alert-warning">Optimizations to do:<ul><li>Image augmentation: not applied on single images but on a batch for better performance.</li><li>Test newly implemented image augmentation methods: rotation range to 360°, shear, random resize crops more balanced, color, blur, contrast</li><li>Image augmentation: Discuss impact of augmentation on the individual MDS dimensions (Dimensions, "lightness", "shape", "organisation" (or a possible dimension "size") impaired by harsh crops, resizings, brightness?).</li><li>Learning rate: Discuss usage of lr scheduler (torch.optim.lr_scheduler)</li></ul></div>

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import datasets, models, transforms, utils
from torchmetrics import R2Score
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import copy
from copy import deepcopy
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import cv2
import PIL
from PIL import Image
from skimage import io, transform
from glob import glob


device = 'cuda' if torch.cuda.is_available() else 'cpu'


# number of models
n_ensemble = 10

# size of test and validation set
n_test = 90

# Number of dimensions in the dataset
n_dim = 8

# Batch size for training (change depending on how much memory you have)
batch_size_im = 90
batch_size_ft = 30

# Number of epochs to train for 
n_epochs_im = 500
n_epochs_ft = 500
early_stop_tolerance = 40

# where would we use these in pytorch? in keras they are used to create layers for the intermediate model
dropout = 0.5 
n_dense = 256
n_layers = 2

# Flag for feature extracting. When False, we finetune the whole model, 
#   when True we only update the reshaped layer params
feature_extract_im = True
feature_extract_ft = False

input_size = 224

loglr = -2.2200654426745987
lr_im = 1 * 10 ** loglr
lr_ft = 1.5e-3 # 2e-3 and 1.5e-3 best so far

# create categories of stones for stratified train test split
categories = [i for i in range(30) for j in range(12)] # creates 360 list items like so: [0, 0, 0, 0, ... 29, 29, 29, 29]

IMG_360 = '../sanders_2018/360 Rocks/'
IMG_120 = '../sanders_2018/120 Rocks/'
MDS_360 = '../finetuning_torchvision_data/mds_360.csv'
CHECKPOINTS = 'CNN_checkpoints/'
PATH_IM = CHECKPOINTS + 'intermediate_model.pt'
PATH_FT = CHECKPOINTS + 'finetuned_model.pt'


print("Device is", device)

Device is cuda


## Functions, classes

In [2]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=14, is_inception=False, early_stop_tolerance=20):
    """
    handles the training and validation of a given model. At the end of
    training returns the best performing model. After each epoch, the training and validation
    accuracies are printed
    """
    
    since = time.time()

    val_acc_history = []
    lrs = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = None
    
    r2score = R2Score(num_outputs=8).to(device)
    
    early_stopping = EarlyStopping(early_stop_tolerance)
    
    for epoch in range(num_epochs):
        print(f'Epoch {str(epoch + 1).rjust(len(str(num_epochs)))}/{num_epochs}', end=": ")
        #print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_r2 = 0.0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    score = r2score(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        # scheduler.step()
                        lr = optimizer.param_groups[0]["lr"]
                        lrs.append(lr)

                # statistics 
                running_loss += loss.item() * inputs.size(0)
                running_r2 += score.item() * inputs.size(0)
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset) 
            epoch_acc = running_r2 / len(dataloaders[phase].dataset) 
            # print only if phase is validation
            if phase == "val":
                print(f'Loss = {epoch_loss: .4f}, Score = {epoch_acc: .4f}, lr = {lr: .4e}')
                early_stopping(epoch_loss)

            # deep copy the model
            if phase == 'val' and best_acc == None:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val' and best_acc < epoch_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        # early stopping if no improvement in the last 20 epochs
        if early_stopping.early_stop:
            print("Early stopped.")
            break
                

        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, lrs


def set_parameter_requires_grad(model, feature_extracting):
    """
    This helper function sets the ``.requires_grad`` attribute of the
    parameters in the model to False when we are feature extracting.
    """
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    else:
        for param in model.parameters():
            param.requires_grad = True
    return None


class RocksData(datasets.VisionDataset):
    def __init__(self, df, root_dir, transform=None):
        super(RocksData).__init__()
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, ix):
        img_path = self.root_dir + "/" + self.df.iloc[ix,0]
        img = cv2.imread(img_path)
        img = PIL.Image.fromarray(img)  
        label = deepcopy(self.df.iloc[ix,1:].tolist())
        label = torch.tensor(label).float()
        img = self.transform(img)
        return img.to(device), label
 
    
class PredictionData(datasets.VisionDataset): # TODO: lots of redundant code. integrate this into above RocksData dataset
    def __init__(self, img_paths):
        super(PredictionData).__init__()
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std=[0.229, 0.224, 0.225])
        self.img_paths = img_paths
    def __len__(self): return len(self.img_paths)
    def __getitem__(self, ix):
        img = cv2.imread(self.img_paths[ix])/255.
        img = self.preprocess_input(img)
        return img
    def preprocess_input(self, img):
        img = cv2.resize(img, (224,224))
        img = torch.tensor(img).permute(2,0,1)
        img = self.normalize(img).float()
        return img.to(device) 
    
    
def get_criterion(loss_name):
    """
    Returns the optimizer
    """
    if loss_name == "L1":
        return torch.nn.L1Loss()
    elif loss_name == "L2":
        return torch.nn.MSELoss()
    elif loss_name == "smooth_L1":
        return torch.nn.SmoothL1Loss()
    elif loss_name == "huber":
        return torch.nn.HuberLoss()
    else:
        raise Exception("No valid loss_name entered!")
        

def load_pretrained_model(stage="intermediate", e=None, model_type=None):
    
    if (model_type == None) | (model_type == "resnet"):
        model = models.resnet50()
    elif model_type == "regnet":
        model = models.regnet_y_16gf()
    set_parameter_requires_grad(model, feature_extract_ft)
    num_ftrs = model.fc.in_features
    new_layers = OutputLayers(n_layers, num_ftrs, n_dense, dropout)
    model.fc = new_layers # replace last layer
    
    if model_type == None:
        if e == None:
            checkpoint = torch.load(CHECKPOINTS + f'{stage}_model.pt', map_location=device)
        else:
            checkpoint = torch.load(CHECKPOINTS + f'{stage}_model_{e}.pt', map_location=device)
    else:
        checkpoint = torch.load(CHECKPOINTS + f'{stage}_{model_type}.pt', map_location=device)
    
    model.load_state_dict(checkpoint)

    # Send the model to GPU
    return model.to(device)


def predict(model, data_loader, unlabeled=False):
    """Computes predictions for a given mnodel and dataset"""
    
    model.eval()

    outputs = list()
    since = time.time()
    with torch.no_grad():
        if unlabeled:
            for inputs in data_loader:
                inputs = inputs.to(device)
                output = model(inputs)
                outputs.append(output.to("cpu").squeeze().numpy())
        else:
            for inputs, _ in data_loader:
                inputs = inputs.to(device)
                output = model(inputs)
                outputs.append(output.to("cpu").squeeze().numpy())

    return np.array(outputs)


class EarlyStopping():
    def __init__(self, tolerance=20):

        self.tolerance = tolerance
        self.counter = 0
        self.early_stop = False
        self.best_loss = np.nan

    def __call__(self, val_loss):
        if val_loss >= self.best_loss:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

## Prepare Data

### Define image transforms

<div class="alert alert-warning">Optimizations to be done here. See dimensions organization and shape</div>

old

In [3]:
# define image transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size, scale=(1, 1), ratio=(.33, 2.5)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomAffine(degrees=360, translate=(0.2, 0.2), shear=(-16, 16, -16, 16), scale=(0.8, 1.4)), 
#        transforms.ColorJitter(brightness=(.95, 1.05), contrast=(.9, 1.1), saturation=(0.95, 1.15)), 
        transforms.ColorJitter(brightness=(.9, 1.10), contrast=(.85, 1.15), saturation=(0.95, 1.15)), # new
        transforms.GaussianBlur(kernel_size=(1, 5), sigma=(.01, .5)), # new
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),    
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

new

### Create dataset and dataloaders

In [4]:
df = pd.read_csv(MDS_360)


# split data: train vs test
(train, test,
categories_train_, categories_test) = train_test_split(df,
                                                       categories,
                                                       test_size=n_test,
                                                       stratify=categories,
                                                       random_state=0)

# split train set again: train vs validate
train, val = train_test_split(train,
                              test_size=n_test,
                              stratify=categories_train_, 
                              random_state=0)

# create image datasets
train_dataset = RocksData(train.reset_index(drop=True), root_dir=IMG_360, transform=data_transforms["train"])
val_dataset = RocksData(val.reset_index(drop=True), root_dir=IMG_360, transform=data_transforms["val"])
test_dataset = RocksData(test.reset_index(drop=True), root_dir=IMG_360, transform=data_transforms["val"])
pred_dataset = PredictionData(glob(IMG_120+'*.jpg'))

## Adapt model

This is how the final layers look like in Keras:

```
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
==================================================================================================

avg_pool (GlobalAveragePooling  (None, 2048)        0           ['conv5_block3_out[0][0]']       
 2D)                                                                                              
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['avg_pool[0][0]']               
                                                                                                  
 dense (Dense)                  (None, 256)          524544      ['dropout[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 256)         1024        ['dense[0][0]']                  
 alization)                                                                                       
                                                                                                  
 dropout_1 (Dropout)            (None, 256)          0           ['batch_normalization[0][0]']    
                                                                                                  
 dense_1 (Dense)                (None, 256)          65792       ['dropout_1[0][0]']              
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 256)         1024        ['dense_1[0][0]']                
 rmalization)                                                                                     
                                                                                                  
 dropout_2 (Dropout)            (None, 256)          0           ['batch_normalization_1[0][0]']  
                                                                                                  
 dense_2 (Dense)                (None, 8)            2056        ['dropout_2[0][0]']       
```

In [5]:
from collections import OrderedDict

class OutputLayers(nn.Sequential):
    def __init__(self, n_layers, num_ftrs, n_dense, dropout):
        super().__init__(self.init_modules(n_layers, num_ftrs, n_dense, dropout))


    def init_modules(self, n_layers, num_ftrs, n_dense, dropout):
        modules = OrderedDict()
        
        i = 0
        modules[f"dropout_{i}"] = nn.Dropout(p=dropout)
        modules[f"fc_{i}"] = nn.Linear(num_ftrs, n_dense)
        
        for i in range(1, n_layers):           
            modules[f"relu_{i}"] = nn.ReLU(inplace=True)
            modules[f"batchnorm_{i}"] = nn.BatchNorm1d(n_dense)
            modules[f"dropout_{i}"] = nn.Dropout(p=dropout)
            modules[f"fc_{i}"] = nn.Linear(n_dense, n_dense)

        modules[f"relu_{i+1}"] = nn.ReLU(inplace=True)
        modules[f"batchnorm_{i+1}"] = nn.BatchNorm1d(n_dense)
        modules[f"dropout_{i+1}"] = nn.Dropout(p=dropout)
        modules[f"fc_{i+1}"] = nn.Linear(n_dense, n_dim)

        return modules

## Train one model ...
... for testing purposes, comparing learning rates etc.

Intermediate model

## Train ensemble

Changes 11.07.2022
* wider range of color changes
* same settings as on 10.07.
* r = .667441
* best model in 3 dims
* the "weak" dimensions improved visibly, especiall dimension 5, chromacity

Changes 10.7. 19:39
* added gaussian blur
* r 0.670374
* best model in 3 dims

Changes 10.7. 10:00
* back to old transforms configuration
* lr 1.5e-3
* num_early_stop = 40
* r = .666

Changes 10.7. 9:32
* lr 2e-3 instead of 1.5e-3
* poor

Changes 9.7. 22:00
* RandomResizeCrop scale and ratio to default values (0.08, 1) aund (.75, 1.33)
* RandomAffine scale to default value None
* r 0.669
* 4 wins
* dimension 1 could be better

Changes 9.7.
* Shear from 20 to 16
* scale=(.7, 1.5) instead of (1, 1) in RandomResizeCrop
* r 0.664

Changes 9.7.
* Shear change from 24 to 20
* r .669
* not the best, dimension 1 could have been better

Changes 8.7. 23:00
* Shear increased from 16 to 24
* R²s 62, 73.4
* r .662
* not bad, but does not come too close to best results

Changes 8.7. 22:00
* adding gaussian blur again
* overall not really better, maybe a little worse, but
* seems more stable in the problematic dimensions shape and organization
* worth further tests

Changes 8.7. 21:19
* interpolate method in randomaffine back to nearest
* good again

Changes 8.7. 20:42
* Also set RandomAffine interpolate methode to bilinear
* Results a little worse
* 63.9, 73

Changes 8.7. 19:32
* set RandomResizeCrop to interpolate bilinear instead of nearest
* results are close to the best

Changes 8.7. 18:43
* bring back scale in randomaffine
* set back scale and ratio in randomresizecrop to previous
* no real changes im 61, ft 72

Changes 8.7., 18:10:
* Shear set back from 9 to 16
* better: im: 60, ft: 71

Changes 8.7., 17:30
* GaussianBlur removed
* Poor: R² intermediate: .56, fintetuned: .69

Changes 8.7., 17:00
* new augmention techniques
  * RandomAffine: smaller shear, no scale, translate increased from .16 to .20
  * GaussianBlur added again
  * ColorJitter: all transforms slightly increased
  * RandomResizeCrop: with scale, different ratio, different interpolation (nearest)
* lr Adagrad set to 1.5e-3
* NEW INTERMEDIATE models generated here
* aborted after 1 model with R²=.72 and r=.65


optimal learning rate is betweend 1.5e-3 and 2e-3. best looking results so far with 2e-3, best mean correlation at 1.5e-3

Changes 8.7., 10:00
* lr Adagrad = 1.8e-3 instead of 1.5e-3
* r=0.666998
* fast

Changes 8.7., 6:00
* lr Adagrad = 1.5e-3 instead of 2e-3 => slightly better overall, dimension 1 not
* r=0.667969

Changes 7.7. 19:00
* lr Adagrad = 5e-3 => poor, high loss
* lr Adagrad = 5e-4 => poor results
* lrAdagrad = 2e-3 => best results so far + very fast!

Changes 7.7. 16:28:
* lr Adagrad = 1e-4 instead of 1e-3
=> poor results R² = .67

Changes 6.7.:
* new INTERMEDIATE model for each finetuned model
* no GaussianBlur
* early stop at 25 instead of 20 epochs without loss reduction


In [ ]:
new_im_each_epoch = True # False means using the same intermediate model for all ensemble models


for e in range(1, n_ensemble + 1):
    
       
    # each epoch a new intermediate model?
    if new_im_each_epoch:
        
        # check if model for this epoch already exists
        intermediate_available = os.path.isfile(CHECKPOINTS + f"intermediate_model_{e}.pt")
        
        if intermediate_available == False:

            # Intermediate model
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            set_parameter_requires_grad(model, feature_extract_im)
            num_ftrs = model.fc.in_features
            new_layers = OutputLayers(n_layers, num_ftrs, n_dense, dropout)
            model.fc = new_layers # replace last layer

            # Send the model to GPU
            model = model.to(device)

            # create datalaoders with specific batch size
            train_loader = DataLoader(train_dataset, batch_size=batch_size_im)
            val_loader = DataLoader(val_dataset, batch_size=batch_size_im)
            dataloaders_dict = {"train":train_loader,"val":val_loader}

            # Create Optimizer and define params to update
            params_to_update = model.parameters()
            if feature_extract_im:
                params_to_update = []
                for name,param in model.named_parameters():
                    if param.requires_grad == True:
                        params_to_update.append(param)
                        #print("\t",name)

            else:
                for name,param in model.named_parameters():
                    if param.requires_grad == True:
                        #print("\t",name)
                        ...

            # Instantiate optimizer for intermediate model
            optimizer = optim.Adam(params_to_update, lr=lr_im)

            # Setup the loss fxn
            criterion = get_criterion('L2')

            # Initial training and evaluate
            model, hist, lrs = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=n_epochs_im, early_stop_tolerance=early_stop_tolerance)

            # Plot learning curve
            plt.plot(hist)
            plt.title(f'Intermediate model {e}')
            plt.show()

            # Save intermediate model
            PATH_EN = f'CNN_checkpoints/intermediate_model_{e}.pt'
            torch.save(model.state_dict(), PATH_EN)
            del model
        model = load_pretrained_model("intermediate", e=e)
        
    else: # load a standard intermediate model
        model = load_pretrained_model("intermediate")

    # fine tuning
    
    # create dataloaders with specific batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size_ft)
    val_loader = DataLoader(val_dataset, batch_size=batch_size_ft)
    dataloaders_dict = {"train":train_loader,"val":val_loader}
            
    # Create Optimizer and define params to update
    params_to_update = model.parameters()
    if feature_extract_ft:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                #print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                #print("\t",name)
                ...

    # Instantiate optimizer for finetuning
    # optimizer = optim.SGD(params_to_update, lr=lr_ft, momentum=0.9)
    optimizer = optim.Adagrad(params_to_update, lr=lr_ft)
   
    # Setup the loss fxn
    criterion = get_criterion("L2")

    # Train and evaluate fine tuned model
    model, hist, lrs = train_model(model, dataloaders_dict, criterion, optimizer, num_epochs=n_epochs_ft, early_stop_tolerance=early_stop_tolerance)
    
    # Plot learning curve
    plt.plot(hist)
    plt.title(f'Ensemble model {e}')
    plt.show()
    
    # Save intermediate model
    PATH_EN = f'CNN_checkpoints/ensemble_model_{e}.pt'
    torch.save(model.state_dict(), PATH_EN)

Epoch   1/500: Loss =  6.7413, Score = -0.0221, lr =  6.0247e-03
Epoch   2/500: Loss =  6.3673, Score =  0.0317, lr =  6.0247e-03
Epoch   3/500: Loss =  5.7737, Score =  0.1148, lr =  6.0247e-03
Epoch   4/500: Loss =  5.1823, Score =  0.1984, lr =  6.0247e-03
Epoch   5/500: Loss =  4.7794, Score =  0.2577, lr =  6.0247e-03
Epoch   6/500: Loss =  4.4776, Score =  0.3008, lr =  6.0247e-03
Epoch   7/500: Loss =  4.2026, Score =  0.3393, lr =  6.0247e-03
Epoch   8/500: Loss =  3.9993, Score =  0.3657, lr =  6.0247e-03
Epoch   9/500: Loss =  3.8097, Score =  0.3890, lr =  6.0247e-03
Epoch  10/500: Loss =  3.6363, Score =  0.4116, lr =  6.0247e-03
Epoch  11/500: Loss =  3.4958, Score =  0.4308, lr =  6.0247e-03
Epoch  12/500: Loss =  3.3561, Score =  0.4515, lr =  6.0247e-03
Epoch  13/500: Loss =  3.2597, Score =  0.4660, lr =  6.0247e-03
Epoch  14/500: Loss =  3.1502, Score =  0.4828, lr =  6.0247e-03
Epoch  15/500: Loss =  3.0475, Score =  0.4991, lr =  6.0247e-03
Epoch  16/500: Loss =  2.

## Load checkpoints and get predictions for validation and test sets

### Create dataloaders for inference

In [7]:
pred_loader = DataLoader(pred_dataset, batch_size=1, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

### Run ensemble models (CNNs) ...
... to collect their individual predictions

In [8]:
test_pred_ = []
rocks_120_pred_ = []

#n_ensemble = 4

for e in range(1, n_ensemble+1):
    model = load_pretrained_model("ensemble", e)
    pred = predict(model, test_loader)
    test_pred_.append(pred)
    rocks_120_pred_.append(predict(model, pred_loader, unlabeled=True))
    
del model

test_pred = np.mean(test_pred_, 0)
rocks_120_pred = np.mean(rocks_120_pred_, 0)

rocks_120_pred_ = np.array(rocks_120_pred_)

### Get MSE

In [9]:
# get labels
Y_120 = np.loadtxt("../sanders_2018/mds_120.txt")
Y_validate = train.iloc[:, 1:].values
Y_test = test.iloc[:, 1:].values

In [10]:
print(mean_squared_error(Y_test, test_pred)) # same MDS space
print(mean_squared_error(Y_120, rocks_120_pred)) # not part of MDS space

1.5717692361821305
2.4398613737882897


### Get R²

In [11]:
print(r2_score(Y_test, test_pred))
print(r2_score(Y_120, rocks_120_pred))

0.7309429938627188
-0.16887360765449788


### Get Pearson's *r* for invidivual dimensions in Rocks 120 set

* Best overall pytorch10 ensemble was from 2022-07-08 with lr=1.5e-3 => 0.667969
* second best from 2022-07-12 lr=1.5e-3 r=.667441 (with gaussian blur and wider color spectrum) 
* third best overall pytorch 10 model ensemble was from 2022-07-08 with lr=2e-3 mean correlation 0.667118 (custom model 0.670676)

In [34]:
m_i = [4, 4, 4, 9, 2] # add models to a custom ensemble

###

rocks_120_pred_custom = np.mean(rocks_120_pred_[[i-1 for i in m_i], :, :], 0)

cnn_pred_file = "CNN Predictions/MDS Dimensions/cnn_predicted_mds_120.txt"
cnn_pred = np.loadtxt(cnn_pred_file)

cnn_keras_pred_file = "CNN Predictions/MDS Dimensions/cnn_keras_predicted_mds_120.txt"
cnn_keras_pred = np.loadtxt(cnn_keras_pred_file)

df_corr = pd.DataFrame(index=range(1,9))

for dim in range(1, 9):
    df_corr.loc[dim, "Keras Sanders & Nosofsky"] = np.corrcoef(cnn_pred[:, dim-1], Y_120[:, dim-1]).min()
    df_corr.loc[dim, "Keras replicated"] = np.corrcoef(cnn_keras_pred[:, dim-1], Y_120[:, dim-1]).min()
    df_corr.loc[dim, "PyTorch ensemble"] = np.corrcoef(rocks_120_pred[:, dim-1], Y_120[:, dim-1]).min()
    df_corr.loc[dim, "PyTorch custom"] = np.corrcoef(rocks_120_pred_custom[:, dim-1], Y_120[:, dim-1]).min()
    for i in range(len(rocks_120_pred_)):
        df_corr.loc[dim, f"PyTorch {i+1}"] = np.corrcoef(rocks_120_pred_[i, :, dim-1], Y_120[:, dim-1]).min()
    
df_corr.loc["Total", :] = df_corr.mean(axis=0)
df_corr["Best model"] = df_corr.columns[df_corr.values.argmax(axis=1)]

df_corr.T[["Total"]].drop("Best model", axis=0).sort_values("Total", ascending=False)

,Total
Keras replicated,0.683814
Keras Sanders & Nosofsky,0.676178
PyTorch custom,0.674779
PyTorch 4,0.670695
PyTorch ensemble,0.667441
PyTorch 9,0.664012
PyTorch 2,0.662079
PyTorch 8,0.661162
PyTorch 1,0.656888
PyTorch 5,0.653353


In [35]:
df_corr

,Keras Sanders & Nosofsky,Keras replicated,PyTorch ensemble,PyTorch custom,PyTorch 1,PyTorch 2,PyTorch 3,PyTorch 4,PyTorch 5,PyTorch 6,PyTorch 7,PyTorch 8,PyTorch 9,PyTorch 10,Best model
1,0.879750,0.877838,0.864368,0.867425,0.821212,0.855097,0.834189,0.863502,0.860229,0.855371,0.836939,0.860703,0.858924,0.872101,Keras Sanders & Nosofsky
2,0.850301,0.826120,0.836048,0.836445,0.812192,0.827837,0.814159,0.829510,0.817364,0.830978,0.825905,0.836433,0.833779,0.822286,Keras Sanders & Nosofsky
3,0.378968,0.358630,0.374706,0.376814,0.344754,0.372719,0.392444,0.369428,0.367931,0.358049,0.391602,0.341089,0.381092,0.362156,PyTorch 3
4,0.838155,0.831899,0.846803,0.847684,0.839813,0.832114,0.839642,0.844954,0.844309,0.838581,0.825956,0.829436,0.835461,0.817548,PyTorch custom
5,0.484210,0.548242,0.448063,0.500841,0.448021,0.422874,0.462506,0.531739,0.420590,0.349347,0.411128,0.456019,0.452347,0.423682,Keras replicated
6,0.841105,0.857191,0.880103,0.882565,0.896047,0.847970,0.873678,0.883627,0.844866,0.851555,0.875491,0.883320,0.876183,0.866906,PyTorch 1
7,0.477049,0.496715,0.403270,0.414652,0.409390,0.435216,0.334163,0.413083,0.386381,0.379384,0.423798,0.407796,0.365063,0.363499,Keras replicated
8,0.659885,0.673881,0.686167,0.671803,0.683676,0.702808,0.666104,0.629718,0.685152,0.672955,0.632572,0.674503,0.709245,0.631178,PyTorch 9
Total,0.676178,0.683814,0.667441,0.674779,0.656888,0.662079,0.652111,0.670695,0.653353,0.642027,0.652924,0.661162,0.664012,0.644920,Keras replicated


### Save predictions to file

In [14]:
np.savetxt("CNN Predictions/MDS Dimensions/cnn_torch_ensemble10_22-07-12_predicted_mds_120.txt", rocks_120_pred)

---
### Idea of using a meta learner (LinearRegression, ElasticNet or similar) ...
... to find optimally weighted mean of CNN predictions.

$Y_{pred} =  b_0 + b_1 Y_{pred_1} + b_2 Y_{pred_2}$

<div class="alert-warning"><ul><li>According to their paper Sanders & Nosofsky were not using a meta-learner, they were training an ensemble of 10 models and averaging them.</li><li>Making a custom ensemble by adding only a selection of all trained models and adding some models twice or more times we can quickly see that weights can improve the performance. However, the differences with this simple method were only marginal compared to the differences that we see between models on certain dimensions such as shape or organization. Thus, the latter issue should be tackled first.</ul></div>